In [99]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 29, in <module>
  File "main.py", line 29, in <dictcomp>
KeyboardInterrupt: 
MicroPython 94a9b50-dirty on 2022-03-20; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [100]:
from lib.runner import Runner
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

Nc = 1
Ns = 50

stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

gc.collect()

calibration_data = {}

import ujson as json
from ulab import numpy as np

data = requests.GETRequest("http://192.168.0.37:5001/calib_1_7")
calibration_data[7] = np.array(json.loads(data)['7'])

gc.collect()

data = requests.GETRequest("http://192.168.0.37:5001/calib_1_10")
calibration_data[10] = np.array(json.loads(data)['10'])

gc.collect()

data = requests.GETRequest("http://192.168.0.37:5001/calib_1_12")
calibration_data[12] = np.array(json.loads(data)['12'])

runner.calibrate(calibration_data)

# calibration_data = #import json saved on ROM
# runner.calibrate(calibration_data)
gc.collect()
    

connecting to network...
.network config: ('192.168.0.88', '255.255.255.0', '192.168.0.1', '194.168.4.100')
ADC initialised
SPI initialised
DigiPot set to 255 = gain of 17.57861814046062


In [110]:
for count in range(26):
    test = requests.GETRequest("http://192.168.0.37:5001/test_7_1",{'count':count})
    test = np.array(json.loads(test)['test'])
    print(runner.decoder.classify(test))

{12: 0.353692236735091, 10: 0.1684230887620481, 7: 0.3213921672094082}


In [71]:
import ujson as json
print(json.loads(test))

{'7': [[-4.39086, -1.517698, -2.567706, -0.9628060000000001, -0.35166, -6.026739999999999, 5.372126, 3.1714366, -4.069384, -5.93382, -2.073244, 0.220948, -5.424259999999999, -8.522459999999999, -4.64647, -3.13742, 2.3625, 2.942354, 1.251274, -3.7493158, 2.13166, -3.8074, -0.722114, 5.58588, 4.181900000000001, 3.2131, 5.337072, -1.8962294, -3.51056, -0.09456000000000007, 4.66236, 3.95916, 0.1104499999999999, 1.886844, -0.91294, -4.27594, 4.172680000000001, 1.4868, 2.284958, 5.54468, -0.7515880000000001, -1.221656, 0.512116, 6.877660000000001, 3.130644, 3.94192, 3.900416, -5.9847, -3.2161, -0.2231540000000002], [-0.5553999999999998, -5.620659999999999, -3.380139999999999, -5.270160000000001, -4.79872, 1.59459, -2.706598000000001, 0.16388, 13.31864, 8.232339999999999, -5.060239999999999, 6.168646, 2.31429, 0.2420080000000001, -1.838674, 1.65052, -1.985932, 0.64594, 1.14461, -2.6032, 3.613, 5.77638, -5.384506, -7.10416, -5.14444, -2.44932, 5.008519999999999, 1.093496, 6.65436, -1.560442000